# Manual Live BusObservatory Data Lake Compaction

In [1]:
system_id = "TEST_tfnsw_bus"
# system_id = "TEST_njtransit_bus"

In [2]:
import os
os.environ["AWS_PROFILE"] = "remote_notebook_user"

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

block_size = str(1024 * 1024 * 128)

conf = (SparkConf()
        .set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.profile.ProfileCredentialsProvider')
        .set("spark.hadoop.dfs.block.size", block_size)
        .set('spark.driver.port', '7077')
        .set('spark.driver.cores', '2')
        .set('spark.driver.memory', '4g')
        .set('spark.executor.memory', '2g')
       )

spark = (SparkSession
         .builder
         .appName('BusObservatory-manual-compaction')
         .config(conf=conf)
         .getOrCreate()
        )
spark.sparkContext.uiWebUrl

22/08/29 10:12:04 WARN Utils: Your hostname, marseilles.local resolves to a loopback address: 127.0.0.1; using 10.0.0.30 instead (on interface en0)
22/08/29 10:12:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/anthonytownsend/.ivy2/cache
The jars for the packages stored in: /Users/anthonytownsend/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c78b9aba-234f-4059-828a-59d0ea69ee95;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/bin/mambaforge/envs/bigdata/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 116ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-c78b9aba-234f-4059-828a-59d0ea69ee95
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/8ms)
22/08/29 10:12:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your pla

'http://10.0.0.30:4041'

In [4]:
#TODO update for main data lake
# bucket = “busobservatory”
bucket = "busobservatory-migration"
in_path = f"s3a://{bucket}/{system_id}/*"
# out_path = f"s3a://{bucket}/{system_id}-compacted/"
out_path = f"s3a://{bucket}/{system_id}/compacted/"

In [5]:
import boto3
import math

def get_folder_size(bucket, path):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)
    total_size = 0
    count_obj = 0
    for obj in my_bucket.objects.filter(Prefix=path):
        total_size = total_size + obj.size
        count_obj = count_obj + 1

    return total_size, count_obj

def get_repartition_factor(dir_size):
    # block_size = sc._jsc.hadoopConfiguration().get("dfs.blocksize")
    block_size = int(spark._jsc.hadoopConfiguration().get("dfs.blocksize"))
    return math.ceil(dir_size/block_size)

In [6]:
dir_size, count_obj = get_folder_size(bucket, system_id)
num_files = get_repartition_factor(dir_size)
block_size = int(spark._jsc.hadoopConfiguration().get("dfs.blocksize"))
print(f'Compacting {dir_size/1024/1024:.1f} Mb from {count_obj} files into {num_files} files of {block_size/1024/1024:.1f} Mb each.')

Compacting 1.4 Mb from 49 files into 1 files of 128.0 Mb each.


In [ ]:
input_df=spark.read.csv("<hdfs_directory>").\
withColumn("filename",input_file_name())

In [7]:
# if in_path and out_path are the same, this overwrites the data before it can read it all

df = (spark.read
      .format("parquet")
      .load(in_path)
      .repartition(num_files)
      .write
      .option("dataChange", "false")
      .format("parquet")
      .mode("overwrite")
      .save(out_path))

22/08/29 10:12:07 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/08/29 10:12:08 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
22/08/29 10:12:08 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
22/08/29 10:12:13 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 49)   
java.io.FileNotFoundException: 
No such file or directory: s3a://busobservatory-migration/TEST_tfnsw_bus/compacted/part-00000-d1f1f677-b166-4b83-99ec-28c8d598565f-c000.snappy.parquet

It is possible the underlying files have been up

Py4JJavaError: An error occurred while calling o68.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at org.ap

In [ ]:
out_dir_size, out_count_obj = get_folder_size(bucket, f'{system_id}-compacted')
in_dir_size, in_count_obj = get_folder_size(bucket, system_id)
num_files = get_repartition_factor(in_dir_size)
block_size = int(spark._jsc.hadoopConfiguration().get("dfs.blocksize"))
print(f'Compaction complete: {out_dir_size/1024/1024:.1f} Mb in {out_count_obj-1} files is near optimal ({num_files} files of {block_size/1024/1024:.1f} Mb each.)')

In [ ]:
# TODO delete files read in?
